# Plotly/Dashによるインタラクティブ・データ可視化入門ハンズオン
## Dash編 / 2020/12/5

- Dashの概要を解説し、最終的にアプリケーションを作成する
- なるだけ手を動かしてもらう
- 今回はjupyter_dashを使う     
  - dash / ウェブアプリケーションを作成する
  - jupyter_dash / jupyter上で動作するアプリケーションを作成する
    - jupyter_dashの注意点：jupyter上で動作するため、データ分析をするときに使えて便利だったりするが、公開されて時間がたっていないため、うまく動かない理由が分からないことや、そもそも動かない（dashでは動いても）というケースもあるのでちょっと注意を。今回使うのはハンズオンには便利だからです。


In [ ]:
# まずはjupyter_dashパッケージのインストールとplotlyのアップグレード
!pip install jupyter_dash 
!pip install plotly --upgrade

     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 1.0MB 19.1MB/s 
     |████████████████████████████████| 3.5MB 47.4MB/s 
     |████████████████████████████████| 194kB 47.6MB/s 
     |████████████████████████████████| 1.8MB 43.8MB/s 
     |████████████████████████████████| 358kB 31.5MB/s 
  Created wheel for dash: filename=dash-1.17.0-cp36-none-any.whl size=83728 sha256=47c34b734e2070159f1dcb360ad176787dd0b31525bc2331d4faf02c75a64dfb
  Stored in directory: /root/.cache/pip/wheels/e0/32/13/8a569b638a79cf80a4a721f56040d91198adec28e3bc8123fc
  Created wheel for dash-renderer: filename=dash_renderer-1.8.3-cp36-none-any.whl size=1013944 sha256=59d24e5b9bbdbc88e12ecd6f66885bbd3a5758bf89fc1687ec4a4ef349be8306
  Stored in directory: /root/.cache/pip/wheels/cf/2b/5f/0928a6d1b7ebf280f21a2e925f36d662e6ba83e00b82c6b6bf
  Created wheel for dash-core-components: filename=dash_core_components-1.13.0-cp36

In [ ]:
from jupyter_dash import JupyterDash 
import dash_core_components as dcc 
import dash_html_components as html 
from dash.dependencies import Input, Output

import pandas as pd
import plotly.express as px

In [ ]:
# ドロップダウンの選択によりグラフの種類が更新されるアプリケーション

app = JupyterDash(__name__)

graph_type = [px.line, px.scatter, px.bar]

app.layout = html.Div([
                       dcc.Dropdown(id='my_drop', options=[{'label': type_.__name__, 'value': num}
                                    for num, type_ in enumerate(graph_type)], value=0),
                       dcc.Graph(id='my_graph')
])

@app.callback(Output('my_graph', 'figure'), Input('my_drop', 'value'))
def update_graph(drop_value):
  return graph_type[drop_value](x=[1,2,3,4,5], y=[2,3,5,3,2], title=f'{graph_type[drop_value].__name__}グラフ')

app.run_server(mode='inline')



<IPython.core.display.Javascript object>

In [ ]:
import  datetime as dt 

In [ ]:
# python3.9ならzoneinfoが使えたのに・・・
now = dt.datetime.now(tz=dt.timezone(dt.timedelta(hours=9)))
time(now.hour, now.minute, now.second)

datetime.time(22, 12, 14)

In [ ]:
sec_oneday = pd.date_range('2020-11-1', '2020-11-2', freq='1S')

df = pd.DataFrame()
df['dt'] = sec_oneday 
col_names = ['a', 'b', 'c', 'd', 'e']

for col in col_names:
  df[col] = df[col].map(100)

In [ ]:
df

,dt,a,b,c,d,e
0,2020-11-01 00:00:00,0,0,0,0,0
1,2020-11-01 00:00:01,0,0,0,0,0
2,2020-11-01 00:00:02,0,0,0,0,0
3,2020-11-01 00:00:03,0,0,0,0,0
4,2020-11-01 00:00:04,0,0,0,0,0
...,...,...,...,...,...,...
86396,2020-11-01 23:59:56,0,0,0,0,0
86397,2020-11-01 23:59:57,0,0,0,0,0
86398,2020-11-01 23:59:58,0,0,0,0,0
86399,2020-11-01 23:59:59,0,0,0,0,0
